In [ ]:
'''
서울시 내에서 스타벅스 매장이 어떻게 분포되어 있는지 확인
일반 매장과, 리저브, 드라이브스루 매장 분포에 어떤 차이가 있는지
* 가설1 : 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2 : 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다. 
'''

## 01. 데이터 시각화

### 01-1. 스타벅스 매장분포 시각화

In [8]:
import pandas as pd
import folium
import json

# 서울시 스타벅스 매장 목록 데이터 불러오기
seoul_starbucks = pd.read_excel('./files/seoul_starbucks_list.xlsx')
seoul_starbucks.head()

매장명         위도          경도         매장타입                          주소  \
0  역삼아레나빌딩  37.501087  127.043069  pin_general     서울특별시 강남구 언주로 425 (역삼동)   
1   논현역사거리  37.510178  127.022223  pin_general    서울특별시 강남구 강남대로 538 (논현동)   
2  신사역성일빌딩  37.513931  127.020606  pin_general    서울특별시 강남구 강남대로 584 (논현동)   
3   국기원사거리  37.499517  127.031495  pin_general    서울특별시 강남구 테헤란로 125 (역삼동)   
4  대치재경빌딩R  37.494668  127.062583  pin_reserve  서울특별시 강남구 남부순환로 2947 (대치동)   

        전화번호 시군구명  
0  1522-3232  강남구  
1  1522-3232  강남구  
2  1522-3232  강남구  
3  1522-3232  강남구  
4  1522-3232  강남구

In [9]:
# folium을 이용한 지도 생성
starbucks_map = folium.Map(
    location=[37.573050, 126.979189],
#  tiles='Stamen Terrain',
    zoom_start=11
)
starbucks_map

In [10]:
# 스타벅스 매장 위치를 나타내는 서클 마커 그리기
for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']
    
    folium.CircleMarker(
        location=[lat, lng],
        fill = True,
        fill_color='green',
        fill_opacity=1,
        color='yellow',
        weight=1,
        radius=3
    ).add_to(starbucks_map)

starbucks_map

# 결과 : 대체로 스타벅스 매장이 강남권, 중구, 종로구에 많이 분포되어 있음

* 스타벅스 매장 타입별로 색상을 다르게 표시

In [11]:
seoul_starbucks['매장타입'].unique()

array(['pin_general', 'pin_reserve', 'pin_generalDT', 'pin_generalWT'],
      dtype=object)

In [12]:
seoul_starbucks['매장타입'].value_counts()

매장타입
pin_general      545
pin_reserve       39
pin_generalDT     23
pin_generalWT      2
Name: count, dtype: int64

In [17]:
# 스타벅스 매장 타입별 위치 서클 마커 그리기
starbucks_map2 = folium.Map(
    location=[37.573050, 126.979189],
# tiles='Stamen Terrain',
    zoom_start=11
)

# 스타벅스 매장 종류 : general(일반매장), reserve(리저브매장), generalDT(드라이브스루)
# for idx, seoul_starbuck in enumerate(seoul_starbucks)
for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']
    store_type = seoul_starbucks.loc[idx, '매장타입']
    
    fillColor = ''
    if store_type == 'pin_general':
        fillColor = 'purple'
        size = 5
    elif store_type == 'pin_reserve':
        fillColor = 'blue'
        size = 5
    elif store_type == 'pin_generalDT':
        fillColor = 'red'
        size = 5
    elif store_type == 'pin_generalWT':
        fillColor = 'green'
        size = 20
        
    folium.CircleMarker(
        location=[lat, lng],
        color=fillColor,
        fill = True,
        fill_color = fillColor, 
        fill_opacity = 1,
        weight = 1,
        radius = size
    ).add_to(starbucks_map2)

starbucks_map2    

# circle마커 : 위치파악 가능, 데이터가 많아지면 시각화가 어려움, 행정 구역간의 비율 확인이 어려움
        

### 01-2. 시군구별 스타벅스 매장 수 시각화

In [19]:
import pandas as pd
import folium
import json

# 서울시 시군구별 통계 데이터 불러오기
seoul_sgg_stat = pd.read_excel('./files/seoul_sgg_stat.xlsx')
seoul_sgg_stat.head()

시군구코드  시군구명         위도          경도  스타벅스_매장수  주민등록인구
0  11320   도봉구  37.665861  127.031767         6  312858
1  11380   은평구  37.617612  126.922700        14  468766
2  11230  동대문구  37.583801  127.050700        10  354884
3  11590   동작구  37.496504  126.944307        11  390377
4  11545   금천구  37.460097  126.900155        13  242467

* 시군구 행정 경계 지도 + 시군구별 스타벅스 매장 수 = 버블 지도

In [21]:
# 서울시 시군구 행정 경계 지도 파일 불러오기
sgg_geojson_file_path = './map/seoul_sgg.geojson'
seoul_sgg_geo = json.load(open(sgg_geojson_file_path, encoding='utf-8'))
print(type(seoul_sgg_geo))
seoul_sgg_geo

<class 'dict'>


{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'SIG_CD': '11320',
    'SIG_KOR_NM': '도봉구',
    'SIG_ENG_NM': 'Dobong-gu',
    'ESRI_PK': 0,
    'SHAPE_AREA': 0.00210990544544,
    'SHAPE_LEN': 0.239901251347},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.019851357, 37.70088490199998],
      [127.02217147700003, 37.69960736799999],
      [127.02341184299996, 37.69995983299998],
      [127.02533791899998, 37.69948105499998],
      [127.02692041600005, 37.70015310500003],
      [127.02704481599994, 37.70101150699998],
      [127.02771004600004, 37.70083744700003],
      [127.02823835, 37.700188083],
      [127.02833534499996, 37.70006885100003],
      [127.02901307699995, 37.69957318899998],
      [127.02930740500005, 37.699210467],
      [127.02962280199995, 37.69810365199999],
      [127.02967567200005, 37.69773805400001],
      [127.02975002699998, 37.

In [22]:
seoul_sgg_geo['features'][0]['properties']

{'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0,
 'SHAPE_AREA': 0.00210990544544,
 'SHAPE_LEN': 0.239901251347}

In [23]:
starbucks_bubble = folium.Map(
    location=[37.573050, 126.979189], tiles ='CartoDB dark_matter' , zoom_start=11 )
starbucks_bubble

In [24]:
# 서울시 시군구 경계 지도 그리기
# style_function : jeojson포맷의 지도 데이터를 지도에 표시할 때 스타일을 지정하는 함수
def style_function(feature):
    return {
        'opacity': 0.7,
        'weight': 1,
        'color': 'white',
        'fillOpacity':0,
        'dashArray': '5, 5',
    }

folium.GeoJson(
    seoul_sgg_geo,
    style_function=style_function
).add_to(starbucks_bubble)

starbucks_bubble

In [26]:
# 서울시 시군구별 스타벅스 평균 매장 수 계산
starbucks_mean = seoul_sgg_stat['스타벅스_매장수'].mean()
starbucks_mean

24.36

In [28]:
# 스타벅스 매장수에 따른 크기를 버블 지도로 시각화
# 시군구별로 매장수가 평균보다 많은지 적은지를 색상으로 표현

for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lng = seoul_sgg_stat.loc[idx, '경도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']
    
    if count > starbucks_mean:
        fillColor = 'red'
    else:
        fillColor = 'blue'
    
    folium.CircleMarker(
        location=[lat, lng], 
        color='#FFFF00',
        fill_color=fillColor, 
        fill_opacity=0.7,
        weight=1.5,
        radius=count/2
    ).add_to(starbucks_bubble)
    
starbucks_bubble

# 결과 : 강남구, 서초구, 솓ㅇ파구와 중구, 종로구, 영등포구, 마포구, 서대문구에 스타벅스 매장이 많이 입점해 있음


* 스타벅스 매장수 단계구분도 시각화

In [29]:
sgg_geojson_file_path = './map/seoul_sgg.geojson'
seoul_sgg_geo_2 = json.load(open(sgg_geojson_file_path, encoding='utf-8'))
starbucks_choropleth = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB dark_matter',
    zoom_start=11
)

folium.Choropleth(
    geo_data=seoul_sgg_geo_2,
    data=seoul_sgg_stat,
    columns=['시군구명', '스타벅스_매장수'],
    fill_color = 'YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='properties.SIG_KOR_NM'
    ).add_to(starbucks_choropleth)

starbucks_choropleth